## Teleco Customer Churn : 통신회사 고객의 해지율 데이터 
- 개별 고객의 데이터를 살펴보고, 통신회사의 고객에 대한 분석을 하고자 한다. 
- **만약 내가 이 회사에서 일하고 있다면, 가장 궁금한 것이 무엇일까?**
- 열에 대한 설명은 다음과 같다. 총 21개의 열
    - customerID : 개별 사용자 키값
    - gender : 사용자의 성별
    - SeniorCitizen : senior citizen 여부(맞으면 1, 아니면 0), senior citizen에 대한 자세한 정의는 없음. 사전적 정의에 따르면, 60 혹은 65세가 넘은 사람을 의미함
    - Partner : 사용자에게 파트너(배우자)가 있는지 없는지(있으면 yes, 아니면 no)
    - Dependents : 피부양자 여부(있으면 yes, 아니면 no)
    - tenure : 근속연수
    - PhoneService : 사용자가 phone service를 사용하는지에 대한 여부(사용하면 yes, 아니면 no)
    - MultipleLines : 사용자가 여러 개의 phone service를 사용하는지에 대한 여부(여러 개면 Yes,하나만 사용하면 No, 아예 phoneService를 사용하지 않으면 No Phone service)
    - InternetService : 사용자가 사용하는 인터넷 서비스 종류(DSL, Fiber optic,없으면 No)
    - OnlineSecurity : 사용자가 온라인 보안을 사용하는지에 대한 여부 (인터넷 서비스+보안이면 Yes, 인터넷 서비스 + 보안X No, 아예 사용을 하지 않으면 No internet service)
    - OnlineBackup : 사용자가 온라인 백업 서비스를 사용하는지에 대한 여부(인터넷 서비스+백업이면 Yes, 인터넷 서비스 + 백업X No, 아예 사용을 하지 않으면 No internet service)
    - DeviceProtection : 기계보안 사용여부 (인터넷 서비스+기계보안이면 Yes, 인터넷 서비스 + 기계보안X No, 아예 사용을 하지 않으면 No internet service)
    - TechSupport : 기술지원 여부 (인터넷 서비스+기술지원이면 Yes, 인터넷 서비스 + 기술지원X No, 아예 사용을 하지 않으면 No internet service)
    - StreamingTV : 스트리밍 tv 사용여부(인터넷 서비스+스트리밍tv이면 Yes, 인터넷 서비스 + 스트리밍tvX No, 아예 사용을 하지 않으면 No internet service)
    - StreamingMovies : 스트리밍 영화 사용여부(인터넷 서비스+스트리밍Movies이면 Yes, 인터넷 서비스 + 스트리밍MoviesX No, 아예 사용을 하지 않으면 No internet service)
    - Contract : 계약 기간(매월, 1년, 2년)
    - PaperlessBilling : 영수증없이 요금납부 여부(Yes, No)
    - PaymentMethod : 요금납부방법(Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))
    - MonthlyCharges : 매월 요금
    - TotalCharges : 총 요금
    - churn : 해지여부 (Yes, No)

### data load and import packages

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
df = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head() 
#출력이 잘되는지 확인

In [ ]:
df.columns

In [ ]:
#1. 데이터의 정보 확인
df.info()
#null값이 없고, 대부분 object임. info 상에서는 데이터들의 빈 곳이 없어보임

In [ ]:
#원래 숫자여야 하는 값이 object형으로 되어 있음. 숫자형으로 바꿔주기로 함
# df['TotalCharges'].astype(float)
#그러나 다음과 같은 에러가 등장함..!
#이 경우, 실제로 데이터가 dataframe에 다 들어가있는 것처럼 보이지만, 사실은 그렇지 않은 경우도 있음을 시사하는 것이다. 
# info에 의하면 null값이 없다고 했으니, 혹시 공백이 있는지 확인해보기로 한다. 

In [ ]:
df[df['TotalCharges']==' ']
# 공백으로 입력이 되어 있다보니, 이를 다시 숫자로 바꾸지 못하게 된 것이다. 

In [ ]:
#2. 전처리 수행
# df[column] = df[column].apply(lambda x: float(x.split()[0].replace(',', '')))
df['TotalCharges'].replace(to_replace = ' ', value= np.nan, inplace = True) #공백을 그냥 null값으로 대체해준다. 
df['TotalCharges'] = df['TotalCharges'].astype(float) 
# df

In [ ]:
df['month'] = round((df['TotalCharges'])/(df['MonthlyCharges'])) #전체 요금에서 월별 요금을 나누었을 때의 값이다. 
df['month'].describe() #최대 79개월까지 사용한 유저가 있었다. 보통 32개월정도 사용하는 것으로 보이지만 중앙값은 29개월정도임(2년을 조금 넘은 정도임)

In [ ]:
fig = px.histogram(df,x='tenure')
fig.show()
#구간화를 위한 확인 - 근속연수 구간을 기준으로 자르기
#신입 연차와 오래 일한 사람이 많음. 중간 연차는 적음

In [ ]:
df['tenure_cut'] = pd.cut(df.tenure, bins=[0, 10, 30, 50,70], labels=['A', 'B', 'C', 'D'])
# 신입 연차, 중간 연차-1, 중간 연차 -2, 오래 된 연차로 4가지 등급으로 나누기로 함

### 어떤 사람들이 가입을 했는지 EDA

In [ ]:
#3. 사용자의 특성별로 데이터를 나누어서 시각화하여 확인하기로 함

In [ ]:
customer_character = df.groupby(['gender','SeniorCitizen','Partner','Dependents','tenure_cut'])[['customerID']].count()
customer_character.rename(columns = {'customerID':'count'},inplace=True)
customer_character.reset_index(inplace=True)

In [ ]:
customer_character.head()

In [ ]:
#사용자 개인 특성 별로 EDA
fig = px.bar(customer_character,x='gender',y="count",color='Partner')
# fig = px.bar(long_df, x="nation", y="count", color="medal", title="Long-Form Input")
fig.show()

In [ ]:
fig = px.bar(customer_character,x='gender',y="count",color='SeniorCitizen')
fig.show()

In [ ]:
fig = px.bar(customer_character,x='Dependents',y="count",color='Partner')
fig.show()
#보통 피부양자가 없으며,배우자가 없는 경우도 많음. 피부양자가 있는데, 배우자가 없는 경우도 있었음

In [ ]:
customer_service = df.groupby(['PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies'])[['customerID']].count()
customer_service.reset_index(inplace=True)
customer_service.rename(columns={'customerID':'count'},inplace=True)

In [ ]:
customer_contract = df.groupby(['Contract', 'PaperlessBilling',
                                'PaymentMethod', 'MonthlyCharges', 'month'])[['customerID']].count()
customer_contract.reset_index(inplace=True)
customer_contract.rename(columns={'customerID':'count'},inplace=True)

### 어떤 사람들이 해지를 할까?
- 오랫동안 가입한 사람들이 해지를 할까?
- 아니면, 조금 사용해보고 해지를 할까?
- 해지자의 가입개월 수의 통계적으로 유의한 차이가 있는지 확인
- 